# Exploring Flow*

In this blog post we [...]

## Installing the tool

We first download the tool version 2.1.0 (May 2017) from the webpage https://flowstar.org/dowloads/

The installation process is simple provided that the required dependencies are met.

We installed Flow* in two different machines:

- Ubuntu LTS v20.04 running on a Windows desktop machine (Asus ZenBook UX331UA-AS51).
- Dell XPS15 running Linux Fedora 34.

In Fedora, we used `dnf install` to install the required dependencies `gmp-devel`, `gsl-devel`, `glpk-devel`, `flex` and `bison`. The library `mpfr` was downloaded and compiled using the instructions in the [downloads page](https://www.mpfr.org/mpfr-current/).

The required dependencies are detailed in Flow*'s [user manual](https://home.cs.colorado.edu/~xich8622/manual/manual-2.0.0.pdf).

## Solving the Lotka-Volterra model

Apart from the models that can be downloaded in the [Flow* webpage](https://flowstar.org/), we are aware of an online model repository from the [*Theory of Hybrid Systems Group* from RWTH Aachen University](https://ths.rwth-aachen.de/research/projects/hypro/benchmarks-of-continuous-and-hybrid-systems/).

We use the Lotka-Volterra model because it is a simple model. For comparison, the same model can be found in the documentation ReachabilityAnalysis.jl [here](https://juliareach.github.io/ReachabilityAnalysis.jl/dev/models/LotkaVolterra/). 

The model file is copied next. It can be read as a plain text file.

```
continuous reachability
{
 state var x,y
	
 setting
 {
  fixed steps 0.02
  time 5
  remainder estimation 1e-5
  QR precondition
  gnuplot octagon x,y
  adaptive orders { min 4 , max 6 }
  cutoff 1e-20
  precision 53
  output Lotka_Volterra
  print on
 }
	
 poly ode 2
 {
  x' = 1.5*x - x*y
  y' = -3*y + x*y
 }
 
 init
 {
  x in [4.8 , 5.2]
  y in [1.8 , 2.2]
 }
}
```

The model file describes a purely continuous model (`continuous reachability`), then the names for the state variables are defined (`state var x,y`).

The next part defines the settings for reachability analysis (`setting`):

- `fixed steps 0.02`: use fixed time steps of size `0.02`
- `time 5`: time horizon for the simulation
- `remainder estimation 1e-5`: estimate of the remainder before each iteration
- `QR precondition`: preconditioning technique for the Taylor expansion
- `gnuplot octagon x,y`: overapproximation method for plotting
- `adaptive orders { min 4 , max 6 }`: range of Taylor model's order
- `cutoff 1e-20`: threshold for polynomial coefficients (
- `precision 53`: precision (number of bits) for the interval bounds
- `output Lotka_Volterra`: number of output file
- `print on`: verbose on

The following block of code defines the differential equations; in this case,

```
 poly ode 2
 {
  x' = 1.5*x - x*y
  y' = -3*y + x*y
 }
```

Where `poly ode 2` specifies a system with two variables.

Finally, the `init` block specify the settings for reachability analysis. 

```julia
 init
 {
  x in [4.8 , 5.2]
  y in [1.8 , 2.2]
 }
```

In this case this defines a hyperrectangle of sides `4.8 .. 5.2` and `1.8 .. 2.2`.

## Running the example

We made a small modification in the model that can be downloaded from 

[ ..... ]

## Parsing the result

In [60]:
src = "LotkaVolterra/outputs/Lotka_Volterra.flow"

"LotkaVolterra/outputs/Lotka_Volterra.flow"

In [72]:
text = String(read(src))
flow = split(text, "{");

flow = flow[3:end]

N = length(flow) # number of reach-sets

500

In [73]:
R = flow[1] # first reach-set
data = split(R, "\n")

12-element Vector{SubString{String}}:
 ""
 "x = [5.00000000000000e0 , 5.00" ⋯ 2446 bytes ⋯ "0625e-8 , 1.70313210117098e-8]"
 ""
 "y = [2.00000000000000e0 , 2.00" ⋯ 2452 bytes ⋯ "2587e-8 , 1.77595808728572e-8]"
 ""
 ""
 "local_t in [0 , 1.00000000000001e-2]"
 "local_var_1 in [-1.00000000000000e0 , 1.00000000000000e0]"
 "local_var_2 in [-1.00000000000000e0 , 1.00000000000000e0]"
 "}"
 ""
 ""

In [80]:
data_x = data[2] # "x" variable

data_x = replace(data_x, "," => "..")
data_x = replace(data_x, "[" => "(")
data_x = replace(data_x, "]" => ")")
data_x = split(data_x, "=")[2] |> strip

data_x = replace(data_x, "local_var_1" => "x")
data_x = replace(data_x, "local_var_2" => "y")
data_x = replace(data_x, "local_t" => "t")

data_x

"(5.00000000000000e0 .. 5.00000000000000e0) + (2.00000000000000e-1 .. 2.00000000000001e-1) * x + (-2.50000000000000e0 .. -2.50000000000000e0) * t + (-1.00000000000001e0 .. -1.00000000000000e0) * t * y + (-1.00000000000001e-1 .. -1.00000000000000e-1) * t * x + (-9.3750000" ⋯ 1455 bytes ⋯ " (2.30833333333333e-1 .. 2.30833333333335e-1) * t^4 * x^2 + (-8.41718750000027e0 .. -8.41718749999971e0) * t^5 * y + (5.42578124999991e0 .. 5.42578125000012e0) * t^5 * x + (-3.93610026041663e1 .. -3.93610026041660e1) * t^6 + (-1.43728770330625e-8 .. 1.70313210117098e-8)"

The aim is to convert this reach-set to a `TaylorModelN` whose coefficients are intervals.

In [81]:
using TaylorModels

In [82]:
t, x, y = set_variables("t x y", order=8)

3-element Vector{TaylorN{Float64}}:
  1.0 t + 𝒪(‖x‖⁹)
  1.0 x + 𝒪(‖x‖⁹)
  1.0 y + 𝒪(‖x‖⁹)

In [85]:
R = Meta.parse(data_x) |> eval

 [4.99999, 5.00001] + [-2.5, -2.5] t + [0.199999, 0.200001] x + [-9.375, -9.375] t² + [-0.100001, -0.0999999] t x + [-1.00001, -1] t y + [2.39583, 2.39584] t³ + [-1.37501, -1.375] t² x + [-0.500001, -0.5] t² y + [-0.0400001, -0.0399999] t x y + [20.7421, 20.7422] t⁴ + [-0.570834, -0.570833] t³ x + [3.79166, 3.79167] t³ y + [-0.0400001, -0.0399999] t² x² + [-0.120001, -0.119999] t² x y + [0.0999999, 0.100001] t² y² + [7.93619, 7.9362] t⁵ + [4.30885, 4.30886] t⁴ x + [4.67708, 4.67709] t⁴ y + [-0.0933334, -0.0933333] t³ x² + [0.351666, 0.351667] t³ x y + [0.316666, 0.316667] t³ y² + [-0.00400001, -0.00399999] t² x² y + [0.00399999, 0.00400001] t² x y² + [-39.3611, -39.361] t⁶ + [5.42578, 5.42579] t⁵ x + [-8.41719, -8.41718] t⁵ y + [0.230833, 0.230834] t⁴ x² + [1.28499, 1.28501] t⁴ x y + [-0.345834, -0.345833] t⁴ y² + [-0.00266667, -0.00266666] t³ x³ + [0.00133333, 0.00133334] t³ x² y + [0.0393333, 0.0393334] t³ x y² + [-0.00666667, -0.00666666] t³ y³ + 𝒪(‖x‖⁹)

In [86]:
typeof(R)

TaylorN{Interval{Float64}}

- Taylor1
- TaylorN
- TaylorModel1
- TaylorModelN

In [55]:
aux = split(data_x, "+")
pol = aux[1:end-1]
rem = aux[end]

" (-3.71654142564716e-6 .. 3.61155996611113e-6)"

In [39]:
using Symbolics, IntervalArithmetic

In [40]:
@variables local_t local_var_1 local_var_2

3-element Vector{Num}:
     local_t
 local_var_1
 local_var_2

In [47]:
f = eval(Meta.parse(data_x));

In [16]:
expr = :([5.00000000000000e0 , 5.00000000000000e0] + [2.00000000000000e-1 , 2.00000000000001e-1] * local_var_1 + [-2.50000000000000e0 , -2.50000000000000e0] * local_t + [-1.00000000000001e0 , -1.00000000000000e0] * local_t * local_var_2 + [-1.00000000000001e-1 , -1.00000000000000e-1] * local_t * local_var_1 + [-9.37500000000000e0 , -9.37500000000000e0] * local_t^2 + [-4.00000000000002e-2 , -4.00000000000000e-2] * local_t * local_var_1 * local_var_2 + [-5.00000000000001e-1 , -5.00000000000000e-1] * local_t^2 * local_var_2 + [-1.37500000000001e0 , -1.37500000000000e0] * local_t^2 * local_var_1 + [2.39583333333333e0 , 2.39583333333334e0] * local_t^3 + [1.00000000000000e-1 , 1.00000000000001e-1] * local_t^2 * local_var_2^2 + [-1.20000000000001e-1 , -1.20000000000000e-1] * local_t^2 * local_var_1 * local_var_2 + [-4.00000000000002e-2 , -4.00000000000000e-2] * local_t^2 * local_var_1^2 + [3.79166666666666e0 , 3.79166666666668e0] * local_t^3 * local_var_2 + [-5.70833333333335e-1 , -5.70833333333333e-1] * local_t^3 * local_var_1 + [2.07421874999999e1 , 2.07421875000001e1] * local_t^4 + [4.00000000000000e-3 , 4.00000000000002e-3] * local_t^2 * local_var_1 * local_var_2^2 + [-4.00000000000002e-3 , -4.00000000000000e-3] * local_t^2 * local_var_1^2 * local_var_2 + [3.16666666666666e-1 , 3.16666666666668e-1] * local_t^3 * local_var_2^2 + [3.51666666666667e-1 , 3.51666666666668e-1] * local_t^3 * local_var_1 * local_var_2 + [-9.33333333333337e-2 , -9.33333333333334e-2] * local_t^3 * local_var_1^2 + [4.67708333333331e0 , 4.67708333333336e0] * local_t^4 * local_var_2 + [4.30885416666666e0 , 4.30885416666669e0] * local_t^4 * local_var_1 + [7.93619791666664e0 , 7.93619791666665e0] * local_t^5 + [-3.71654142564716e-6 , 3.61155996611113e-6])

:([5.0, 5.0] + [0.2, 0.200000000000001] * local_var_1 + [-2.5, -2.5] * local_t + [-1.00000000000001, -1.0] * local_t * local_var_2 + [-0.100000000000001, -0.1] * local_t * local_var_1 + [-9.375, -9.375] * local_t ^ 2 + [-0.0400000000000002, -0.04] * local_t * local_var_1 * local_var_2 + [-0.500000000000001, -0.5] * local_t ^ 2 * local_var_2 + [-1.37500000000001, -1.375] * local_t ^ 2 * local_var_1 + [2.39583333333333, 2.39583333333334] * local_t ^ 3 + [0.1, 0.100000000000001] * local_t ^ 2 * local_var_2 ^ 2 + [-0.120000000000001, -0.12] * local_t ^ 2 * local_var_1 * local_var_2 + [-0.0400000000000002, -0.04] * local_t ^ 2 * local_var_1 ^ 2 + [3.79166666666666, 3.79166666666668] * local_t ^ 3 * local_var_2 + [-0.570833333333335, -0.570833333333333] * local_t ^ 3 * local_var_1 + [20.7421874999999, 20.7421875000001] * local_t ^ 4 + [0.004, 0.00400000000000002] * local_t ^ 2 * local_var_1 * local_var_2 ^ 2 + [-0.00400000000000002, -0.004] * local_t ^ 2 * local_var_1 ^ 2 * local_var_2 + [0.

In [12]:
flow[4]

"\nx = [4.94627245832176e0 , 4.94627245832178e0] + [-2.01689183333334e-2 , -2.01689183333333e-2] * local_var_2 + [1.97446122750000e-1 , 1.97446122750001e-1] * local_var_1 + [-2.87145614659829e0 , -2.87145614659826e0] * local_t + [4.25333333333333e-5 , 4.25333333333335e-5]" ⋯ 8676 bytes ⋯ "4684265708e-4 , 3.28024684265733e-4] * local_t^4 * local_var_1^2 + [-1.03260271052719e-5 , 1.05744350065855e-5]\n\n\nlocal_t in [0 , 2.00000000000001e-2]\nlocal_var_1 in [-1.00000000000000e0 , 1.00000000000000e0]\nlocal_var_2 in [-1.00000000000000e0 , 1.00000000000000e0]\n}\n\n"

In [13]:
flow[5]

"\nx = [4.88520694988594e0 , 4.88520694988596e0] + [-4.05462410145678e-2 , -4.05462410145675e-2] * local_var_2 + [1.93774983568780e-1 , 1.93774983568782e-1] * local_var_1 + [-3.23311485601497e0 , -3.23311485601492e0] * local_t + [1.79313394630744e-4 , 1.79313394630746e-4]" ⋯ 9948 bytes ⋯ "9198636271e-3 , 1.64169198636279e-3] * local_t^4 * local_var_1^2 + [-1.79237441919779e-5 , 1.82141163949649e-5]\n\n\nlocal_t in [0 , 2.00000000000001e-2]\nlocal_var_1 in [-1.00000000000000e0 , 1.00000000000000e0]\nlocal_var_2 in [-1.00000000000000e0 , 1.00000000000000e0]\n}\n\n"

In [56]:
using TaylorModels

3-element Vector{TaylorN{Float64}}:
  1.0 t + 𝒪(‖x‖⁵)
  1.0 x + 𝒪(‖x‖⁵)
  1.0 y + 𝒪(‖x‖⁵)

In Flow*, the reach-sets are represented using Taylor models in N+1 variables, where the last variable represents "time".

The text file `Lotka_Volterra.flow` contains the output generated by the algorithm.

Let's analyze in more detail the contents of this file.

The first part defines the symbols used for the state variables, in this case `x` and `y`. Also defined is the order of the Taylor models (in this case, `6`), the cutoff threshold (in this case, `1e-20`), and finally the models' name (in this case, `Lotka_Volterra`).

Header of `Lotka_Volterra.flow`:

```
state var x,y

gnuplot octagon x , y

order 6

cutoff 1.000000e-20

output Lotka_Volterra
```